In [102]:
import numpy as np
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [163]:
# アナライザーのフィルター
token_filters = [
    POSKeepFilter(['名詞'])  # 名詞を抽出するようにする
]
analyzer = Analyzer(token_filters=token_filters)

'''
文字列を形態素解析する
    documents 文字列
'''
def wakachi(text):
    result = []
    for tok in analyzer.analyze(text):
        result.append(tok.surface)
    return result

'''
文字列配列をベクトル化する
    documents 文字列配列
'''
def vecs_array(documents):
    docs = np.array(documents)
    vectorizer = TfidfVectorizer(analyzer=wakachi,binary=True,use_idf=True)
    vecs = vectorizer.fit_transform(docs)
    return vecs.toarray()

'''
行列の指定列をベクトル化する
    datas           データ行列
    x_index         基準となるデータの行番号
    vecs_col_index  ベクトル化する列番号
'''
def convert_vecs_col(datas, x_index, vecs_col_index):
    
    # ベクトル化する文書の列取得
    col = datas[:,col_num]

    # ベクトル化
    vecs_col = vecs_array(col)

    # ベクトル同士の類似度計算
    cs_col = np.round(cosine_similarity(vecs_col, vecs_col), 3)

    # 入力値を基準とした類似度取得
    x_vecs = cs_col[x_index]

    # 文書列を類似度に入れ替え
    datas = np.delete(datas, vecs_col_index, 1)
    datas = np.insert(datas, vecs_col_index, x_vecs.T, 1)
    
    return datas

In [179]:
x_index = 1

# CSVから取得したデータ
y_array = np.array([[1, 2, "杉野あきよし", "ケインズ"], [2, 7, "杉野アキヨシ", "けいんず"], [2, 7, "スギノあきよし", "KEINS"]])
#print(y_array)

# 2列目ベクトル化
y_array2 = convert_vecs_col(y_array, x_index, 2)
#print(y_array2)

# 3列目ベクトル化
y_array3 = convert_vecs_col(y_array2, x_index, 3)
#print(y_array3)

# コサイン類似度計算
x_data = np.array([y_array3[x_index]])
cos_sim = cosine_similarity(x_data, y_array3)
print(cos_sim)

[[0.98253971 1.         0.98969335]]
